In [ ]:
import numpy as np
from parse_lsp_data import *
import tensorflow as tf

In [ ]:
from keras.preprocessing import image
from keras import applications
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.models import Model,Input
from keras.utils import np_utils
from keras.models import Sequential
from keras.callbacks import CSVLogger
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout,Reshape
from keras.optimizers import SGD,Adam
import keras.backend as K
from keras.layers import Input, Dense, Dropout, Conv2D, MaxPooling2D, Flatten,Activation, Average,GlobalAveragePooling2D
from image_noise_generation import *

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
# get the data set
train_list, train_label, val_list, val_label, test_list, test_label = getLSPDataset(0.8, 0.1)

In [ ]:
# print some information
train_X = train_list	# (N, 227, 227, 3)
train_Y = train_label	# (N, 2, 14)
train_X,train_Y = data_augmentation(train_X,train_Y)
print("train X: ", train_X.shape, "val X: ", val_list.shape, "test X: ", test_list.shape)
print("train Y: ", train_Y.shape, "val Y: ", val_label.shape, "test Y: ", test_label.shape)

In [ ]:
# model_input = Input(shape=(227,227,3))

In [ ]:
# build keras model
def build_model():
    base_model = applications.densenet.DenseNet121(weights='imagenet', include_top=False, input_shape=(227,227, 3))
    last = base_model.output
    x = Flatten()(last)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.9)(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.9)(x)
    x = Dense(28, activation='relu')(x)
    preds = Reshape((2,14))(x)
    model = Model(base_model.input, preds)
    return model

In [ ]:
model = build_model()

In [ ]:
#model.summary()

In [ ]:
# custom loss function (L2 distance)
def euclidean_distance_loss(y_true, y_pred):
    return K.sqrt(K.sum(K.square(y_pred - y_true)))/64

In [ ]:
thresh = 0.5
def accuracy(y_true, y_pred):
    
    diff = K.square(y_true - y_pred)
#     loss = K.sum(diff)
    dist = K.sqrt(K.sum(diff, axis = 1))	# (N, 2, 14) -> (N, 14)
#     accuracy = tf.reduce_sum(tf.to_int32(tf.greater(1.0, dist))) / 8/14
    torsor_xy = y_true[:,:,9] - y_true[:,:,2] # (N,2)
    torsor_dist = K.sqrt(K.sum(K.square(torsor_xy),axis=1,keepdims=True)) #(N,1)
    torsor_frac = thresh*torsor_dist
    
    accuracy = K.sum(K.cast(K.greater(torsor_frac,dist),dtype='float32'))/64/14
    
    return accuracy

In [ ]:
# compile the model
model.compile(optimizer=Adam(lr=0.00005), loss=euclidean_distance_loss, metrics=[accuracy])

In [ ]:
csv_logger = CSVLogger('densenet121_t05_lrxingxing_dp09.csv', append=True, separator=';')

In [ ]:
with tf.device('/device:GPU:0'):
    # fit the model
    history = model.fit(train_X, train_Y, epochs=100,batch_size=64, validation_data = (val_list,val_label),callbacks=[csv_logger])

In [ ]:
# evaluation
with tf.device('/device:GPU:0'):
    loss,acc = model.evaluate(test_list, test_label,batch_size=64)

In [ ]:
print(loss)
print(acc)

In [ ]:
with tf.device('/device:GPU:0'):
    pred = model.predict(test_list)

In [ ]:
print(pred[0])
print(test_label[0])

In [ ]:
dist = K.sqrt(K.sum(K.square(pred[0] - test_label[0]), axis=0))

In [ ]:
threshold = np.array(1.0)

In [ ]:
temp = tf.reduce_sum(tf.to_int32(tf.greater(threshold, dist)))

In [ ]:
print(temp.shape)

In [ ]:
model.save('densenet121_t05_lrxingxing_dp09.h5')

In [ ]:
# load model
def load_trained_model(weights_path):
    model = build_model()
    model.load_weights(weights_path)
    return model

In [ ]:
model = load_trained_model('densenet121_t05_lrxingxing_dp09.h5')

In [ ]:
model.compile(optimizer=Adam(lr=0.00005), loss=euclidean_distance_loss, metrics=[accuracy])

In [ ]:
pred = model.predict(test_list)

In [ ]:
# pred2 = train_model.predict(train_list)
pred2 = model.predict(train_list)

In [ ]:
# pred3 = train_model.predict(val_list)

In [ ]:
# usage:
# drawLines(val_list[5].copy(),val_label[5].copy(),pred3[5].copy())

In [ ]:
from drawLines_v2 import drawLines
# usage:
index = 6
drawLines(test_list[index].copy(),test_label[index].copy(),pred[index].copy())

In [ ]:
# usage:
index2 = 0
drawLines(train_list[index2].copy(),train_label[index2].copy(),pred2[index2].copy())

In [ ]:
# load model
def load_trained_model(weights_path):
    model = build_model()
    model.load_weights(weights_path)
    return model

In [ ]:
model = load_trained_model('resnet50_t05_lr00005.h5')
model.compile(optimizer=Adam(lr=0.0005), loss=euclidean_distance_loss, metrics=[accuracy])

In [ ]:
# cascade
with tf.device('/device:GPU:0'):
    pred = model.predict(train_list)

In [ ]:
with tf.device('/device:GPU:0'):
    pred_val = model.predict(val_list)

In [ ]:
from crop_image_keras import images_crop

In [ ]:
crop_image_matrix,actual_reshape_size_matrix,scaled_preds_label,scaled_trues_label = images_crop(train_list.copy(),pred.copy(),train_label.copy())

In [ ]:
crop_image_matrix_val,actual_reshape_size_matrix_val,_,_ = images_crop(val_list.copy(),pred_val.copy(),val_label.copy())

In [ ]:
print(crop_image_matrix.shape)
print(actual_reshape_size_matrix.shape)
print(scaled_preds_label.shape)
print(scaled_trues_label.shape)
print(train_list.shape)
print(pred.shape)
print(train_label.shape)

print(crop_image_matrix_val.shape)
print(actual_reshape_size_matrix_val.shape)

print(actual_reshape_size_matrix[0])
print(actual_reshape_size_matrix_val[0])
print(scaled_preds_label[0,0])
print(scaled_trues_label[0,0])

In [ ]:
# # just a quick check my scale computation
# print(train_label[0])
# # print(pred[0])
# print(actual_reshape_size_matrix[0])
# # print(scaled_preds_label[0,0])
# print(scaled_trues_label[0,0])

In [ ]:
# build keras model
def build_cascade_model():
    base_model = applications.resnet50.ResNet50(weights='imagenet', include_top=False, input_shape=(227,227, 3))
    last = base_model.output
    x = Flatten()(last)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.7)(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.7)(x)
    x = Dense(2, activation='relu')(x)
    preds = Reshape((2,1))(x)
    model = Model(base_model.input, preds)
    return model

In [ ]:
# custom loss function (L2 distance)
def euclidean_distance_loss(y_true, y_pred):
    return K.sqrt(K.sum(K.square(y_pred - y_true)))/64

In [ ]:
train_pred_matrix = np.zeros((1600,2,14))
val_pred_matrix = np.zeros((200,2,14))

In [ ]:
# put model separately and have a test
####################################################

In [ ]:
temp_images0 = crop_image_matrix[:,0]
temp_true_labels0 = scaled_trues_label[:,0,:,0,None]
cascade_model0 = build_cascade_model()
cascade_model0.compile(optimizer=Adam(lr=0.0005), loss=euclidean_distance_loss, metrics=[])

In [ ]:
with tf.device('/device:GPU:0'):
    # fit the model
    history = cascade_model0.fit(temp_images0, temp_true_labels0, epochs=50,batch_size=64)

In [ ]:
# store training prediction
with tf.device('/device:GPU:0'):
    temp_pred2_train0 = cascade_model0.predict(temp_images0) # (1600,2,1)

In [ ]:
# scale back to original images to compute loss
temp_pred2_train0_store = temp_pred2_train0*actual_reshape_size_matrix[:,:,0,None]/227

In [ ]:
train_pred_matrix[:,:,0,None] = temp_pred2_train0_store

In [ ]:
temp_val_images0 = crop_image_matrix_val[:,0]
with tf.device('/device:GPU:0'):
    temp_pred2_val0 = cascade_model0.predict(temp_val_images0) # (200,2,1)

In [ ]:
temp_pred2_val0_store = temp_pred2_val0*actual_reshape_size_matrix_val[:,:,0,None]/227

In [ ]:
val_pred_matrix[:,:,0,None] = temp_pred2_val0_store

In [ ]:
# second joint
index = 1
temp_images1 = crop_image_matrix[:,index]
temp_true_labels1 = scaled_trues_label[:,index,:,index,None]
cascade_model1 = build_cascade_model()
cascade_model1.compile(optimizer=Adam(lr=0.0005), loss=euclidean_distance_loss, metrics=[])

In [ ]:
with tf.device('/device:GPU:0'):
    # fit the model
    history = cascade_model1.fit(temp_images1, temp_true_labels1, epochs=50,batch_size=64)

In [ ]:
# store training prediction
with tf.device('/device:GPU:0'):
    temp_pred2_train1 = cascade_model1.predict(temp_images1) # (1600,2,1)
# scale back to original images to compute loss
temp_pred2_train1_store = temp_pred2_train1*actual_reshape_size_matrix[:,:,index,None]/227
train_pred_matrix[:,:,index,None] = temp_pred2_train1_store
temp_val_images1 = crop_image_matrix_val[:,index]
with tf.device('/device:GPU:0'):
    temp_pred2_val1 = cascade_model1.predict(temp_val_images1) # (200,2,1)
temp_pred2_val1_store = temp_pred2_val1*actual_reshape_size_matrix_val[:,:,index,None]/227
val_pred_matrix[:,:,index,None] = temp_pred2_val1_store

In [ ]:
# third joint
index = 2
temp_images2 = crop_image_matrix[:,index]
temp_true_labels2 = scaled_trues_label[:,index,:,index,None]
cascade_model2 = build_cascade_model()

In [ ]:
cascade_model2.compile(optimizer=Adam(lr=1), loss=euclidean_distance_loss, metrics=[])

In [ ]:
with tf.device('/device:GPU:0'):
    # fit the model
    history = cascade_model2.fit(temp_images2, temp_true_labels2, epochs=50,batch_size=64)

In [ ]:
# store training prediction
with tf.device('/device:GPU:0'):
    temp_pred2_train2 = cascade_model2.predict(temp_images2) # (1600,2,1)
# scale back to original images to compute loss
train_pred_matrix[:,:,index,None] = temp_pred2_train2*actual_reshape_size_matrix[:,:,index,None]/227

temp_val_images2 = crop_image_matrix_val[:,index]
with tf.device('/device:GPU:0'):
    temp_pred2_val2 = cascade_model2.predict(temp_val_images2) # (200,2,1)
val_pred_matrix[:,:,index,None] = temp_pred2_val2*actual_reshape_size_matrix_val[:,:,index,None]/227

In [ ]:
# forth joint
index = 3
temp_images3 = crop_image_matrix[:,index]
temp_true_labels3 = scaled_trues_label[:,index,:,index,None]
cascade_model3 = build_cascade_model()
cascade_model3.compile(optimizer=Adam(lr=0.0005), loss=euclidean_distance_loss, metrics=[])

In [ ]:
with tf.device('/device:GPU:0'):
    # fit the model
    history3 = cascade_model3.fit(temp_images3, temp_true_labels3, epochs=50,batch_size=64,verbose=0)

In [ ]:
# store training prediction
with tf.device('/device:GPU:0'):
    temp_pred2_train3 = cascade_model3.predict(temp_images3) # (1600,2,1)
# scale back to original images to compute loss
train_pred_matrix[:,:,index,None] = temp_pred2_train3*actual_reshape_size_matrix[:,:,index,None]/227

temp_val_images3 = crop_image_matrix_val[:,index]
with tf.device('/device:GPU:0'):
    temp_pred2_val3 = cascade_model3.predict(temp_val_images3) # (200,2,1)
val_pred_matrix[:,:,index,None] = temp_pred2_val3*actual_reshape_size_matrix_val[:,:,index,None]/227

In [ ]:
# fifth joint
index = 3
temp_images4 = crop_image_matrix[:,index]
temp_true_labels4 = scaled_trues_label[:,index,:,index,None]
cascade_model4 = build_cascade_model()
cascade_model4.compile(optimizer=Adam(lr=0.0005), loss=euclidean_distance_loss, metrics=[])

In [ ]:
with tf.device('/device:GPU:0'):
    # fit the model
    history = cascade_model4.fit(temp_images4, temp_true_labels4, epochs=50,batch_size=64,verbose=0)

In [ ]:
# store training prediction
with tf.device('/device:GPU:0'):
    temp_pred2_train4 = cascade_model4.predict(temp_images4) # (1600,2,1)
# scale back to original images to compute loss
train_pred_matrix[:,:,index,None] = temp_pred2_train4*actual_reshape_size_matrix[:,:,index,None]/227

temp_val_images4 = crop_image_matrix_val[:,index]
with tf.device('/device:GPU:0'):
    temp_pred2_val4 = cascade_model4.predict(temp_val_images4) # (200,2,1)
val_pred_matrix[:,:,index,None] = temp_pred2_val4*actual_reshape_size_matrix_val[:,:,index,None]/227

In [ ]:
##################################################################

In [ ]:
# the following code put every model together
epoches = 10
train_pred_matrix = np.zeros((1600,2,14))
val_pred_matrix = np.zeros((200,2,14))
for epoch in range(epoches):
    
    for i in range(14):
        
        # prepare the data
        # crop_image_matrix (1600,14,227,227,3) => (1600,227,227,3) extract all images belong to one specific bounding box
        temp_images = crop_image_matrix[:,i]
        # scaled trues label (1600,14,2,14) => (1600,i,2,i) => (1600,2,1)
        temp_true_labels = scaled_trues_label[:,i,:,i,None]
        
        # create the model
        cascade_model = build_cascade_model()
        
        # compile the model
        cascade_model.compile(optimizer=Adam(lr=0.0005), loss=euclidean_distance_loss, metrics=[])
        
        with tf.device('/device:GPU:0'):
            # fit the model
            history = cascade_model.fit(temp_images, temp_true_labels, epochs=1,batch_size=64)
        
        # store training prediction
        with tf.device('/device:GPU:0'):
            temp_pred2_train = cascade_model.predict(temp_images) # (1600,2,1)
        
        # scale back to original images to compute loss
        temp_train_pred2 = temp_pred2_train*actual_reshape_size_matrix[:,:,i,None]/227
        
        train_pred_matrix[:,:,i,None] = temp_train_pred2
        
        # store validation prediction
        
        temp_val_images = crop_image_matrix_val[:,i]
        with tf.device('/device:GPU:0'):
            temp_pred2_val = cascade_model.predict(temp_val_images) # (200,2,1)
            
        temp_val_pred2 = temp_pred2_val*actual_reshape_size_matrix_val[:,:,i,None]/227
        
        val_pred_matrix[:,:,i,None] = temp_val_pred2
        
        
    temp_train_loss = K.sqrt(K.sum(K.square(train_pred_matrix - train_label)))/1600
        
    print( 'training loss is ', temp_train_loss)
    
    temp_val_loss = K.sqrt(K.sum(K.square(val_pred_matrix - val_label)))/200
            